In [8]:
import pandas as pd
import numpy  as np

import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

In [2]:
features = pd.read_csv('../data/features_plus_descriptions.csv')
features.set_index('Feature Type and Number', inplace=True)
features.drop(['S5', 'D21'], axis=0, inplace=True)
print('number of features:', len(features.index))

number of features: 51


In [3]:
target = ['H0_Best','H1_Best', 'H2_Best', 'H3_Best', 'H4_Best', 'H5_Best']

df = pd.read_csv('../data/multilabel_with_tol_raw_data.csv')
X = df.loc[:, features.index]
y = df[target]
y[:14] # Display the target columns

,H0_Best,H1_Best,H2_Best,H3_Best,H4_Best,H5_Best
0,1,0,0,0,0,0
1,0,1,1,0,1,1
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
5,1,0,0,0,0,0
6,1,0,0,0,0,0
7,1,0,0,0,0,0
8,1,0,0,0,0,0
9,1,0,0,0,0,0


In [9]:
def split_data(X, y):
    """
    Using the random state multiplier 6 (the best one for the saved best random forest model),
    split the dataset into the test and other sets and return the test sets.
    """
    random_state = 431 * 6 # best random state for random forest saved model
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size = 0.2, random_state=random_state)
    preprocessor = StandardScaler()
    preprocessor.fit_transform(X_other)
    preprocessor.transform(X_test)
    return X_test, y_test

In [10]:
X_test, y_test = split_data(X, y)

In [6]:
file = open('../results/RF_models_micro.pkl', 'rb')
grid, X_test, y_test = pickle.load(file)
file.close()

In [12]:
best_model = grid[6]

In [14]:
y_pred = best_model.predict(X_test)
print('f1:', f1_score(y_test, y_pred, average="micro"))

f1: 0.6958538706871558
